# <a id='toc1_'></a>[**MICRO-452 Final Report**](#toc0_)
### <a id='toc1_1_1_'></a>[*Group 45: Anton Balykov, Teo Halevi, Cyprien Lacassagne, Seonmin Park*](#toc0_)

# <a id='toc2_'></a>[Table of Contents](#toc0_)


- [Project Objective](#toc3_)    
- [I. Computer Vision](#toc4_)    
  - [I.1 Introduction](#toc4_1_)    
  - [I.2 Camera Set Up](#toc4_2_)    
  - [I.3 Global Obstables and Thymio Detection](#toc4_3_)    
- [II. Global Navigation](#toc5_)    
  - [II.1 Introduction](#toc5_1_)    
  - [II.2 Path Planning](#toc5_2_)    
  - [II.3 Algorithm](#toc5_3_)    
- [III. Local Navigation](#toc6_)    
  - [III.1 Introduction](#toc6_1_)    
  - [III.1 Local Avoidance](#toc6_2_)    
- [IV. Filtering](#toc7_)    
  - [IV.1 Introduction](#toc7_1_)    
  - [IV.2 Methodology](#toc7_2_)    
  - [IV.3 Dynamics of Thymio](#toc7_3_)    
  - [IV.4 Design and Implementation of Filter](#toc7_4_)    
  - [IV.5 Simulation](#toc7_5_)    
- [Conclusion](#toc8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc3_'></a>[Project Objective](#toc0_)

This project leverages the Thymio robot to explore and integrate key concepts introduced in the Introduction to Mobile Robotics course. We hereby focus on implementing and demonstrating core components: computer vision, global navigation, motion control, local navigation, and filtering. The mission of the robot is to navigate itself from the designated start position to the specified goal position, successfully avoiding fixed obstacles as well as dynamically placed one. Achieving this requires precise motion control, robust obstable detection, and efficient filtering technique. Such practical application of the five above-mentioned elements would reflect our team's comprehensive understanding of mobile robotics principles.

# <a id='toc4_'></a>[I. Computer Vision](#toc0_)

## <a id='toc4_1_'></a>[I.1 Introduction](#toc0_)

Computer vision is the first step in our project, as it is the basic input. To steer the robot in an environment, we need to know the robot's position in that environment, what obstacles are present, where to go, and whether data acquisition is proceeding correctly. This is the purpose of the `ComputerVision` class we have implemented. All this data will then be used to predict the optimal path from the starting position to the goal, avoiding obstacles, and to provide an initial estimate of the position to be given to the Kalmann filter.

We distinguish several parts in this explanation: the general setup, including the camera, the creation of the map, and the creation of markers to detect the robot and the goal. We then talk about detecting obstacles in the map, which the robot must avoid. Finally, the detection of the robot and the goal is a crucial part, which will be done using ArUco markers. 

## <a id='toc4_2_'></a>[I.2 Setup : camera, map, and markers](#toc0_)

The first thing to create is the environment in which our robot will evolve. This environment is made up of delimited obstacles in a free zone for the robot. After several computer vision tests, we decided that our obstacles would be represented by red shapes on a white background. This choice is explained in greater detail in the following section on obstacle detection. Once our map has been created, we need to find a way of finding the robot in the image. To do this, we chose to use the ArUco markers provided by the OpenCV library in Python. These markers are QR-code-like representations with unique identifiers that can be easily detected in space. By defining a marker for the robot, and a marker for the goal, we can extract their respective positions to define the robot's shortest path to the goal.

ArUco markers are defined by two things: a dictionary and an identifier. In our case, we use the dictionary `DICT_6X6_250` and IDs 4 and 8 for the robot and goal respectively. A marker is generated as follows, where `marker_id` is the defined ID of the marker, and `marker_size` is the size of the marker in pixels:
```python
dictionary   = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
marker_image = cv2.aruco.generateImageMarker(dictionary, marker_id, marker_size)
```

To have enough markers to test the robustness of our code, we created and printed a grid of 9 markers, for IDs from 0 to 8.

<img src="report-images/aruco_grid.png" alt="aruco_grid" width="500" class="center"/>

The webcam used for our project has a resolution of 1920 by 1080 pixels. The camera is positioned above and in the center of the card, to minimize image distortion. An example of the setup with map, obstacles, robot, robot and goal markers and camera is shown below.

<img src="report-images/original_frame_from_webcam.jpg" alt="Original frame from the webcam" width="750" class="center"/>

When using a camera, two parameters need to be determined: the camera matrix $C$ and the distortion coefficients $D$. Matrix $C$ contains the camera's intrinsic parameters: focal lengths and optical center, along the $x$ and $y$ axes. These parameters are  denoted by: $f_x$, $f_y$ (focal lengths) and $c_x$, $c_y$ (optical center), which gives us: $$C = \begin{pmatrix} f_x & 0 & c_x \\ 0 & f_y & c_y \\ 0 & 0 & 1 \end{pmatrix}$$

The distorsion matrix $D$ is defined as $D = \begin{pmatrix} k_1 & k_2 & p_1 & p_2 & k_3 \end{pmatrix}$ where the parameters are defined in [Camera Calibration](https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html).

Thw two matrices $C$ and $D$ are found using calibration. The code used for our calibration comes from the *GeeksForGeeks* website, on the [Camera Calibration with Python - OpenCV](https://www.geeksforgeeks.org/camera-calibration-with-python-opencv/) page. The calibration is usually performed using a reference image such as a checkerboard. We used a checkerboard with 9 by 7 inner corners, and with $20$mm square length. By taking several photos from different angles and heights of the checkerboard, we can correct the distorted image. An image of the calibration process is shown below.

<img src="report-images/checkerboard_calibration.png" alt="checkerboard_calibration." width="750" class="center"/>

For our purposes, we have decided to set the origin at the top left corner of the image. To achieve this, the optical centers $c_x$ and $c_y$ were set to $0$. In addition, distortion coefficients were considered negligible when comparing data with and without calibration. Once these matrices are known, the camera is initialized and can be used to detect obstacles and ArUco markers.

## <a id='toc4_3_'></a>[I.3 Detection of global obstacles](#toc0_)

To be able to steer the robot along the shortest path, we need to know its environment. This means knowing the location of global obstacles on the map.
After several tests, we decided to create our obstacles using flat red shapes. This decision is part of our global obstacle detection strategy.

To detect obstacles in the image, the first step is to apply a mask to the image to extract only the red areas. To do this, we work in the HSV domain of the image. We have determined that the different shades of red can belong to two intervals defined by:

In [ ]:
lower_red_1 = np.array([0, 100, 100])
upper_red_1 = np.array([10, 255, 255])

lower_red_2 = np.array([170, 100, 100])
upper_red_2 = np.array([180, 255, 255])

Our function `apply_color_mask(frame, mask_color='n')` creates the image's red masks for each of the two intervals using the `cv2.inRange` verification function. To bring the masks together, we compare the images pixel by pixel using the OR operator. This returns an image with values of $255$ (white) for pixels detected as red, and $0$ (black) everywhere else.  Finally, the mask undergoes the morphological operation `cv2.MORPH_CLOSE` to reunite disjointed red areas or complete black pixels present in white areas.

The result of this red mask is shown below. We recall the original image. Note that only the red shapes are shown in the black and white image.

<img src="report-images/original_frame_from_webcam.jpg" alt="original_frame" width="500" class="center"/>

<img src="report-images/red_masked_frame.jpg" alt="red_masked_frame" width="500" class="center"/>

This implementation allows us to assume that all obstacles in the image are red. This will facilitate their detection and enable ideal robot 
control.

Once the red shapes have been extracted, we are interested in extracting their edges. Indeed, this is the only important information about an obstacle, since the obstacle's edge must not be exceeded. To extract the edges, we use a Canny edge detector implemented in OpenCV:
```python
cv2.Canny(red_mask, 50, 150)
```

<img src="report-images/red_edges_frame.jpg" alt="red_edges_frame" width="500" class="center"/>

The Canny edge detector involves several stages: noise reduction using Gaussian blurring, gradient calculation, non-maximum suppression, and edge tracking by hysteresis. The second and third arguments in `cv2.Canny` represent the lower and upper thresholds for the hysteresis step. Pixels with gradient intensities above the upper threshold are classified as edges, while those below the lower threshold are ignored. Pixels with intensities between the thresholds are included as edges only if they are connected to a strong edge. The limit values found were obtained after several tests to extract edges on different shades of red, and appear to be optimal for our application.

To predict the shortest path between the robot and the goal, we use a visibility graph. To use it, we need to know the corners of each obstacle. Our function `get_corners_and_shape_edges(edges)` use a polygonal approximation of the shapes found. This approximation allows each obstacle to be represented as a polygon, and therefore to have the corners of these polygons. To study each contour separately, we use `cv2.findContours`. 
```python
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
```

Each contour is then approximated as a polygon, with a certain tolerance. Since each shape is approximated by polygons, rounded shapes can also be defined as a list of corners. This is the case for the ellipse in the figures below.

In [ ]:
# Approximate the contour to a polygon
epsilon = 0.02 * cv2.arcLength(contour, True)
approx  = cv2.approxPolyDP(contour, epsilon, True)

# Extract corner point from the approximated polygon
corners = [[int(point[0][0]), int(point[0][1])] for point in approx]

Finally, the list of corners returned by our function will have the form:
```python
[[[a1,b1], [a2, b2], [a3, b3]],                      # x and y coordinate of the first obstacle
 [[x1, y1], [x2, y2], [x3, y3], [x4, y4], [x5, y5]], # x and y coordinate of the second obstacle
 [[o1, p1], [o2, p2], [o3, p3], [o4, p4]]]           # x and y coordinate of the third obstacle
```

The display of detected contours and corners gives the following image.

<img src="report-images/red_corners_obstacles_frame.jpg" alt="red_corners_obstacles_frame" width="500" class="center"/>

And by displaying the corners of the red obstacles in green on the original camera image, we obtain the following visualization:

<img src="report-images/original_frame_with_corners_obstacles.jpg" alt="original_frame_with_corners_obstacles" width="500" class="center"/>

Note that the above functions give the positions of obstacle corners in pixels. These positions will be converted to millimetres to maintain consistency in the project's units. This conversion is performed using ArUco markers.

## <a id='toc4_4_'></a>[I.3 Detection of the robot and the goal](#toc0_)

Global obstacle detection is now implemented. The map in which the robot is moving is therefore known, and obstacles are clearly positioned. The missing step in directing the robot is to know its position, as well as the position of its goal. 

As mentioned in the introduction, we use ArUco markers to identify the robot and the goal. For the following explanations, we'll use the image below with markers 7 (top left), 3 (top right), and 5 (bottom). The photo has been deliberately taken at an angle, with marker 5 blurred to demonstrate the robustness of marker detection.

<img src="report-images/aruco_3markers.jpg" alt="aruco_3markers" width="500" class="center"/>

Basic marker detection is performed by the `cv2.aruco.detectMarkers` function, which takes as arguments the frame to be analyzed, the dictionary used for markers (in our case `DICT_6X6_250`), and initialized parameters. This function returns a list of marker corners, as well as the identifiers of recognized markers.

In the ComputerVision class, the marker detection and marker drawing operation is performed by the following lines, in the function `detect_and_estimate_pose()` and gives the image below:
```python
corners, marker_ids, _ = cv2.aruco.detectMarkers(frame_with_markers, self.aruco_dict, parameters=self.aruco_params)
cv2.aruco.drawDetectedMarkers(frame_with_markers, corners, marker_ids)
```

<img src="report-images/aruco_3markers_with_id.jpg" alt="aruco_3markers_with_id" width="500" class="center"/>

We can see that the markers are detected correctly: they are surrounded by a green border, their ID is shown in blue, and fourth corner of each marker is shown in red.

The positioning of the red corner in the image above gives an indication of the next step in the detection process. The corners of a marker are defined in the following order:
1. Bottom left
2. Bottom right
3. Top right
4. Top left (red corner shown)

where the reference for top, bottom, left, right is given by the conventional reading of the text above the marker.

To extract the position of markers in mm, use the function `cv2.aruco.estimatePoseSingleMarkers` as follows: 
```python
rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners,
                                                      self.ARUCO_MARKER_SIDE_LENGTH,
                                                      self.mtx,
                                                      self.dst)
```

The `self.ARUCO_MARKER_SIDE_LENGTH` parameter defined in class initialization is the length of a marker side in mm. It is used to switch from pixel coordinates to mm coordinates. Note also that the `mtx` and `dst` matrices are used, corresponding respectively to the $C$ and $D$ matrices defined in the calibration explained above.

All this is done by calling the function:
```python
frame_with_markers, marker_ids, rvecs, tvecs, aruco_diagonal_pixels = vision.detect_and_estimate_pose(frame_marker)
```

The result contains two variables `rvecs` (rotation vectors) and `tvecs` (translation vectors).
The rotation vector (`rvecs`) is first converted into a rotation matrix, which provides a representation of 3D orientation. A $4\times 4$ identity matrix is initialized to serve as a transformation matrix, with its top-left $3\times 3$ portion allocated for the rotation information. The function `cv2.Rodrigues()` is then used to convert the rotation vector into a $3\times 3$ rotation matrix.

Next, the $3\times 3$ rotation matrix is converted into a quaternion using the `scipy.spatial.transform.Rotation.from_matrix()` method. A quaternion is a four-dimensional representation $[x, y, z, w]$ of rotation. In the quaternion, $x$, $y$, and $z$ represent the vector part, while $w$ represents the scalar part. Finally, the quaternion is converted to Euler angles using the function `euler_from_quaternion`. This function calculates roll (rotation around the $x$-axis), pitch (rotation around the $y$-axis), and yaw (rotation around the $z$-axis). Among these, the yaw angle is extracted because it represents the rotation of the marker in the $x$-$y$ plane, which corresponds to the marker's orientation relative to the camera.

In [7]:
def process_marker_pose(self, frame, marker_ids, rvecs, tvecs):
    """
    Process pose information for each detected marker and draw on a copy of the original frame.

    original_frame: The frame to process
    marker_ids, rvecs, tvecs: Detected marker details (IDs, rotation vectors, translation vectors)
    mtx, dst: Camera matrix and distortion coefficients

    Returns:
        frame_with_vectors: A new frame with the vectors drawn
        markers_data: Data of the markers (x-position, y-position, yaw-angle for each marker)
    """

    frame_with_vectors = frame.copy()
    markers_data = []

    for i, marker_id in enumerate(marker_ids):
        transform_translation_x = 1000*(tvecs[i][0][0]) # 1000* to have in millimeters.
        transform_translation_y = 1000*(tvecs[i][0][1])
        transform_translation_z = 1000*tvecs[i][0][2]

        rotation_matrix = np.eye(4)
        rotation_matrix[0:3, 0:3] = cv2.Rodrigues(np.array(rvecs[i][0]))[0]
        r = R.from_matrix(rotation_matrix[0:3, 0:3])
        quat = r.as_quat()

        roll_x, pitch_y, yaw_z = self.euler_from_quaternion(quat[0], quat[1], quat[2], quat[3])

        roll_x = math.degrees(roll_x)
        pitch_y = math.degrees(pitch_y)
        yaw_z = -(math.degrees(yaw_z)-90)

        markers_data.extend([[marker_id[0], transform_translation_x, transform_translation_y, yaw_z]])

        cv2.drawFrameAxes(frame_with_vectors, self.mtx, self.dst, rvecs[i], tvecs[i], 0.05)

    return frame_with_vectors, markers_data


def euler_from_quaternion(x, y, z, w):
    """
    Convert a quaternion into euler angles (roll, pitch, yaw)
    roll is rotation around x in radians (counterclockwise)
    pitch is rotation around y in radians (counterclockwise)
    yaw is rotation around z in radians (counterclockwise)
    """

    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll_x = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = np.clip(t2, -1.0, 1.0)
    pitch_y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw_z = math.atan2(t3, t4)

    return roll_x, pitch_y, yaw_z

By calling the function `process_marker_pose` defined above, we obtain the frame with the detected vectors, as well as the coordinates of each marker.

```python
frame_with_vectors, markers_data = vision.process_marker_pose(frame_with_markers, marker_ids, rvecs, tvecs)
```

<img src="report-images/aruco_3markers_with_vectors.jpg" alt="aruco_3markers_with_vectors" width="500" class="center"/>

For this image, the coordinates are as follows, with ID, $x$-coordinate (mm), $y$-coordinate (mm) and yaw-angle (degrees) for each marker.

```python
[[5, 166.79759785976782, 139.66100169531774, 97.87032032706453],
 [7, 148.05928528540807, 66.24491386374977, 120.86243771133647],
 [3, 296.46820843077455, 67.33202952024861, -45.05418817033191]]
```

The yaw angle is defined as 0 when the green vector is vertical, and follows the clockwise direction. $x$ increases when the marker goes to the right of the image, and $y$ increases when the marker goes to the bottom of the image.

Returning to the original image with the Thymio, we can now display all the information: obstacles, obstacle corners, robot and goal.

<img src="report-images/original_frame_from_webcam.jpg" alt="original_frame" width="500" class="center"/>

<img src="report-images/frame_aruco_and_corners.jpg" alt="frame_aruco_and_corners" width="500" class="center"/>


**NOTE: About conversion from pixels to mm**

ArUco marker coordinates are given directly in mm by the function `detect_and_estimate_pose()`.
On the other hand, the coordinates of obstacle corners are given in pixels.

To have mm everywhere and ensure consistency, we need to convert the coordinates of the obstacle corners into mm. This is done by finding the pixel-mm conversion factor. To do this, we need a length that is both known in mm and known in pixels. We use the information contained in the ArUco markers. In fact, the corners list used in the `detect_and_estimate_pose()` function contains the pixel coordinates of the corners of each marker. Taking the first marker detected, we can calculate the length of its diagonal in pixels. With the actual length of the diagonal in mm known, the conversion factor is found and used to convert the coordinates of the obstacle corners into mm:

```python
aruco_diagonal_pixels = math.sqrt((corners[0][0][2][1] - corners[0][0][0][1])**2 + (corners[0][0][2][0] - corners[0][0][0][0])**2)
```

## <a id='toc4_5_'></a>[Detection of kidnapping](#toc0_)

The last part for the `ComputerVision` class is to detect whether the camera has been covered, or whether the robot has been kidnapped and repositioned elsewhere. We rely on the detection of the marker assigned to the robot to create the corresponding function. If the marker assigned to the robot does not belong to the list of detected markers, then the function returns `True` (to say that the robot has been kidnapped).

In [ ]:
def is_camera_covered(self, detected_markers):
    """
    Returns True (kidnapped) if the ARUCO_ROBOT_ID is not detected, False otherwise
    """

    if detected_markers is None:
        return True
    else:
        return self.ARUCO_ROBOT_ID not in detected_markers

# <a id='toc5_'></a>[II. Global Navigation](#toc0_)

## <a id='toc5_1_'></a>[II.1 Introduction](#toc0_)

bla bla bla

## <a id='toc5_2_'></a>[II.2 Path Planning](#toc0_)

bla bla bla

## <a id='toc5_3_'></a>[II.3 Algorithm](#toc0_)

bla bla bla

.

.

.

.


# <a id='toc6_'></a>[III. Local Navigation](#toc0_)

## <a id='toc6_1_'></a>[III.1 Introduction](#toc0_)

bla bla bla

## <a id='toc6_2_'></a>[III.1 Local Avoidance](#toc0_)

bla bla bla

.

.

.

.

# <a id='toc7_'></a>[IV. Filtering](#toc0_)

## <a id='toc7_1_'></a>[IV.1 Introduction](#toc0_)


There is a need for a method that compensates the defect that there is no direct mapping between Thymio's sensors and the values which are used for control. The absolute position of the robot can be obtained directly from the camera and global path planning techniques, but the robot may have low precision and update rate, causing drifts when the velocity or acceleration is integrated. We can address this challenge by implementing Kalman filter as it provides a sound framework to use all available measurements in an optimal way. It relies on a model for the system structure and the effect of disturbances both on the state and on the measurements.

The Kalman filter we implemented estimates the position and speed of the robot when it moves along the planned path. Specifically, we implemented extended Kalman filter to take into account the system involves nonlinear dynamics that would need to be linearized around the current estimate, focusing on local optimality.


## <a id='toc7_2_'></a>[IV.2 Methodology](#toc0_)

The Thymio robot moves at constant speed across a pattern made of black and white stripes of 50 mm, identical to one of the activities found in Exercise 8 of the course. The actual mean velocity of the two wheels and the mean reflected light read by the two ground sensors are sent in an event at a constant rate. The python code receives these values, updates the estimated velocity and position of the robot, and displays them continuously in a plot.


To obtain velocity parameters, we began with estimating Thymio's speed by let it cross seven blocks that are 50 mm each with a sample rate of 0.1 seconds and marked peaks of the ground sensor meaasuremnet indicating that it has crossed evey block. With known and obtained information, we could obtain the conversion factor of 0.4 to conver Thymio's speed to mm/s.

![Find Peaks](find_peaks.png "Fine Peaks")

Now, to obtain speed variance, we observed the data where the robot was moving by plotting the robot's speed and computed average velocity and thus the speed variance by diving the average speed with the robot's speed in mm/s.

![Measured Vel](measured_vel.png "Measured Vel")

With the assumption that half of the variance is cause by the measurement and the other half is caused by the perturbation to the states, we obtain 

$q_v = r_v = 16.08621$ $mm^2/s^2$ $\space$ where $\space$ $\sigma^2_{vel} = 32.17242$ $mm^2/s^2$

Variances on position state and measurement as well as those for the angle for orientation ($\sigma^2_{pos}$ and $\sigma^2_{\theta}$) were an arbitrary value that were tuned manually for the specific Thymio robot that is going to be used for demonstration.

## <a id='toc7_3_'></a>[IV.3 Dynamics of Thymio](#toc0_)

To begin with the implementation, the state space model is

$\hat{x}_{k-1|k-1} =  \begin{bmatrix}
                        x_{pos_{k-1}} \\
                        y_{pos_{k-1}} \\
                        \theta_{k-1} \\
                        v_{left_{k-1}} \\
                        v_{right_{k-1}} \\
                        \end{bmatrix}$

Under control input vector,

$ u =[v_{trans}, v_{rot}] $, $\space$ where $\space$ $v_{tran} = \frac{v_{left} + v_{right}}{2} $ $\space$ and $\space$ $ v_{rot} = \frac{v_{left} - v_{right}}{l_{base}} $

Then, we can see that the robot's new position and orientation are predicted in the following way

$x_{pos_{k}} = x_{pos_{k-1}} + v_{trans_{k}} \cdot \cos(\theta_{k-1}) \cdot dk$

$y_{pos_{k}} = y_{pos_{k-1}} + v_{trans_{k}} \cdot \sin(\theta_{k-1}) \cdot dk$

$\theta_{k} = \theta_{k-1} + v_{rot_{k}} \cdot dk$

## <a id='toc7_4_'></a>[IV.4 Design and Implementation of Filter](#toc0_)



We have introduced that the extended Kalman filter takes into account nonlinear system dynamics in the previous sections. In order to make prediction of the next state, We use a nonlinear motion model $f(x, u)$. That is, we can formulate their relation as:

$\hat{x}_{k|k-1} = f(\hat{x}_{k-1|k-1}, u_k)$

Our $f(x, u)$ is then introduced as

$f(\hat{x}_{k-1|k-1}, u_k) = \begin{bmatrix}
1 & 0 & 0 & -0.5 \cdot \cos(\theta_{k-1}) \cdot dk & -0.5 \cdot \cos(\theta_{k-1}) \cdot dk \\ 
0 & 1 & 0 & -0.5 \cdot \sin(\theta_{k-1}) \cdot dk & -0.5 \cdot \sin(\theta_{k-1}) \cdot dk \\ 
0 & 0 & 1 & -\frac{dk}{2 \cdot l_{\text{base}}} & \frac{dk}{2 \cdot l_{\text{base}}} \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

In this prediction step of the filter, we make a use of the predicted covariance matrix $P_{k-1|k-1}$.

$P_{k-1|k-1} = \begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

as each column corresponds to the state variables, from the first column to the last column, being $x_{pos_{k-1}}$, $y_{pos_{k_1}}$, $\theta_{k-1} $, $v_{left_{k-1}}$, and $v_{right_{k-1}}$, respectively.

The updated (current) covariance matrix is as follows:

$P_{k|k-1} = F_{k}P_{k-1|k-1}F_{k}^{T} + Q_{k}$

where $F_{k}$ is the Jacobian of $f_{k}$ matrix, and $Q_{k}$ is the noise covariance matrix of the state model. The $Q_{k}$ matrix specifically represents the degree or the extend of how much the actual motion of Thymio deviates from its assumed state space model which would be the path that it should follow for our case.

We can write the two matrices as below:

$F_{k|k-1} = 
\begin{bmatrix}
1 & 0 & v_{trans} \cdot \sin(\theta_{k-1}) \cdot dk & -0.5 \cdot \cos(\theta_{k-1}) \cdot dk & -0.5 \cdot \cos(\theta_{k-1}) \cdot dk \\ 
0 & 1 & v_{trans} \cdot \cos(\theta_{k-1}) \cdot dk & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk \\ 
0 & 0 & 1 & -\frac{dk}{2 \cdot l_{base}} & \frac{dk}{2 \cdot l_{base}} \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

$Q_{k|k-1} = \alpha \cdot 
\begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

where $\alpha$ is a scaling factor, which we have initialized as $\alpha =10$, such that 

$Q_{k-1|k-1} = 10 \cdot 
\begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

We set $\alpha$ large enough so that the filter tracks large changes in the wheel sensor measurements more closely as if it was smaller.

In the updating process of the extended Kalman filter, we make a use of the term *innovation* which is the difference between the actual measurements and predicted observations. Let $z_{k}$ be the observation vector and $i_{k}$ be the innovation term. Then, we can compute:

$i_{k} = z_{k} - H_{k}\hat{x}_{k-1}$ 

where $H_{k}$ is the measurement matrix that is used to convert the predicted state estimate at time $k$ into predicted sensor measurements at time $k$.

In this project, we have two cases that can happen. One of them is when Thymio gets information from the camera, we name it as "when it has vision," and the other is when it does not get information from the camera, "when it has no vision," specifically when the camera is covered.

When it has vision, 

$H_{k} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\ 
0 & 1 & 0 & 0 & 0 \\ 
0 & 0 & 1 & 0 & 0 \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

When it does not have vision, 

$H_{k} = \begin{bmatrix}
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

as the robot will have to rely only on its wheel sensor measurements in this scenario.

With the innovation $y_{k}$, we obtain the innovation covariance matrix $S_{k}$ to be used for the computation of Kalman Gain $K_{k}$.

$S_{k} = H_{k}P_{k|k-1}H_{k}^{T} + R_{k}$

where $R_{k}$ is the sensor measurement noise covariance matrix. It also has different dimensionality depending on if Thymio has vision or not.

When it has vision,

$R_{k} = \beta \cdot \begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

When it does not have vision,

$R_{k} = \gamma \cdot \begin{bmatrix}
\sigma^2_{vel} & 0\\ 
0 & \sigma^2_{vel} 
\end{bmatrix}$

with the same logic of $H_{k}$ matrices, and constants $\beta = 1$ and $\gamma = 1$ were defined through trial and error experiments just like $\alpha$ of the $Q_{k}$ matrix.

The Kalman Gain matrix is computed as follows:

$K_{k} = P_{k|k-1}H_{k}^{T}S_{k}^{-1}$

and we can update our estimated state and its covariance matrix by

$\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_{k}i_{k}$ $\space$ and $\space$ $ P_{k|k} = (I - K_{k}H_{k})P_{k|k-1}$ $\space$,  respectively.


The above steps are implemented as a class ***KalmanFilterExtended*** and the class is called in the **main.py** file where all the five components of the project are integrated.

The function ***run_ekf()*** ensures that the extened filter is intiated and the snippet from **main.py** below ensures that the filter is running in a loop throughout the entire mission of the robot.

Additionally, we have set parameters that defines *kidnapping* in terms of the difference in the Thymio's orientation and position exceeding the threshold values. Specifically, we have set the threshold for orientation and position to be 35cm and 60°, and exceeding either (or both) of these values would be considered getting kidnaped.

**Codes to be displayed here **

## <a id='toc7_5_'></a>[IV.5 Simulation](#toc0_)

to be shown later

# <a id='toc8_'></a>[Conclusion](#toc0_)
It has been an incredible journey to embark on this project within the constrained timeline of just three weeks, replete with challenges and invaluable learning opportunities. From the very beginning, we have had to face numerous obstacles, such as technical errors and conceptual hurdles, which have pushed us to refine our problem-solving skills and deepen our understanding of mobile robotics. Despite these challenges, our team has been able to overcome setbacks and achieve results we are proud of through persistence, discipline, and a collaborative spirit.

Working through the complexities of the project was as rewarding as it was demanding. From understanding the theoretical basis to the implementation of practical solutions, each step provided an opportunity to reinforce and showcase our grasp of the principles of mobile robotics. The hands-on nature of the project tested not only our knowledge but also helped us grow as engineers, fostering resilience, adaptability, and creativity in tackling real-world problems.

We would like to express our deep gratitude to everyone who guided and supported us through this journey. We would like to thank Professor Mondada, who inspired us with his experience and vision, and the teaching assistants, who were always ready to help, explain doubts, and give us valuable insights. Their encouragement and mentorship played a very important role in driving the success of our project.

It is not only an academic milestone but also a source of personal and professional growth, and we will be carrying the lessons learned herein as we progress in our future endeavors related to robotics and beyond.